In [ ]:
import datetime

from src.analysis.plot import Plot
from src.backtest import BackTest
from src.benchmark import Benchmark
from src.factor.roe import RoeFactor
from src.fund_universe import SECURITY_SEDOL
from src.market import Market
from src.portfolio import Portfolio
from src.rebalance import Rebalance
from src.security_symbol import SecurityTicker
from src.analysis.var import Var
from src.analysis.risk_breakdown_to_factor import RiskBreakdownToFactor
from src.analysis.monthly_performance import MonthlyPerformance

In [ ]:
start_date = datetime.date(2020, 9, 30)
end_date = datetime.date(2020, 12, 31)
security_universe = SECURITY_SEDOL
rebalance_period = 1
rebalance_interval = "1mo"
# could be EQUAL|MIN_TE|MVO
# rebalance_weight_strategy = "MVO"
Factor = RoeFactor
market = Market(security_universe, start_date, end_date)
benchmark = Benchmark(SecurityTicker("^SPX", "index"), start_date, end_date)
benchmark_performance = benchmark.get_performance()

In [ ]:
### equal weight factor
equal_factor = Factor(security_universe, "long")
equal_portfolio = Portfolio(100.0, start_date, end_date)
equal_factor.set_portfolio_at_start(equal_portfolio)

rebalance = Rebalance(
    rebalance_period,
    equal_portfolio,
    equal_factor,
    benchmark,
    rebalance_interval,
    "EQUAL",
)

backtest = BackTest(equal_portfolio, market, rebalance)
backtest.run()

In [ ]:
# var analysis
var = Var(equal_portfolio.value_book.select("date", "value"))
print("normal distibution:")
display(var.get_normal_distribution_var())
print("imperical value:")
display(var.get_imperical_var())

In [ ]:
rb = RiskBreakdownToFactor(equal_portfolio, benchmark, end_date)
rb.plot_correlation()

In [ ]:
rb.total_risk_breakdown_analysis()

In [ ]:
rb.tracking_error_breakdown_analysis()

In [ ]:
### minimum tracking error factor
min_te_factor = Factor(security_universe, "long")
min_te_portfolio = Portfolio(100.0, start_date, end_date)
min_te_factor.set_portfolio_at_start(min_te_portfolio)

rebalance = Rebalance(
    rebalance_period,
    min_te_portfolio,
    min_te_factor,
    benchmark,
    rebalance_interval,
    "MIN_TE",
)

backtest = BackTest(min_te_portfolio, market, rebalance)
backtest.run()

In [ ]:
### MVO factor
mvo_factor = Factor(security_universe, "long")
mvo_portfolio = Portfolio(100.0, start_date, end_date)
mvo_factor.set_portfolio_at_start(mvo_portfolio)

rebalance = Rebalance(
    rebalance_period,
    mvo_portfolio,
    mvo_factor,
    benchmark,
    rebalance_interval,
    "MVO",
)

backtest = BackTest(mvo_portfolio, market, rebalance)
backtest.run()

In [ ]:
### plot
plot = Plot(
    equal_portfolio,
    min_te_portfolio,
    mvo_portfolio,
    benchmark_performance,
    "SPX",
)
plot.plot_performance()

In [ ]:
plot.plot_total_risk()

In [ ]:
plot.plot_tracking_error()

In [ ]:
import polars as pl

stat1 = MonthlyPerformance().get_annualized_stat(
    mvo_portfolio.value_book.select("date", "value"), benchmark_performance
)
stat2 = MonthlyPerformance().get_annualized_stat(
    min_te_portfolio.value_book.select("date", "value"), benchmark_performance
)
stat3 = MonthlyPerformance().get_annualized_stat(
    equal_portfolio.value_book.select("date", "value"), benchmark_performance
)
stat = pl.concat([stat1, stat2, stat3], how="vertical")
pl.concat(
    [
        pl.DataFrame({"label": ["MVO", "MIN_TE", "EQUAL WEIGHT"]}),
        stat,
    ],
    how="horizontal",
)

In [ ]:
mvo_portfolio.total_risk_df